In [1]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Plots, Printf, IndexedGraphs, Statistics, Random
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using TensorTrains: summary_compact
using SparseArrays;

In [34]:
T = 5
N = 10
seed = 2

c = 2
gg = erdos_renyi(N, c/N; seed)
g = IndexedGraph(gg)

λ_unif = 0.5
ρ_unif = 0.2
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        λ[i] = λ_unif
    end
end
λ = sparse(λ)
ρ = fill(ρ_unif,N)
γ = 0.5

# T = 3
# N = 2
# seed = 6

# A = [0 1; 1 0]
# g = IndexedGraph(A)

# λ_unif = 0.7
# ρ_unif = 0.1
# λ = sparse(λ_unif .* A)
# # λ = sparse([0 0; λ_unif 0])
# ρ = fill(ρ_unif, N)
# γ = 0.5

sis = SIS_heterogeneous(λ, ρ, T; γ);
bp = mpbp(sis);

In [35]:
g.A

10×10 SparseMatrixCSC{Int64, Int64} with 20 stored entries:
 ⋅  ⋅  ⋅  ⋅   2  5  ⋅  ⋅  9   ⋅
 ⋅  ⋅  1  ⋅   ⋅  6  ⋅  ⋅  ⋅   ⋅
 ⋅  1  ⋅  ⋅   3  ⋅  ⋅  ⋅  ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   4  ⋅  ⋅  8  ⋅   ⋅
 2  ⋅  3  4   ⋅  ⋅  7  ⋅  ⋅  10
 5  6  ⋅  ⋅   ⋅  ⋅  ⋅  ⋅  ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅   7  ⋅  ⋅  ⋅  ⋅   ⋅
 ⋅  ⋅  ⋅  8   ⋅  ⋅  ⋅  ⋅  ⋅   ⋅
 9  ⋅  ⋅  ⋅   ⋅  ⋅  ⋅  ⋅  ⋅   ⋅
 ⋅  ⋅  ⋅  ⋅  10  ⋅  ⋅  ⋅  ⋅   ⋅

In [36]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [37]:
X

10×6 Matrix{Int64}:
 1  1  1  1  1  1
 2  2  2  2  2  2
 1  1  2  2  1  1
 2  2  1  1  1  1
 2  2  1  1  1  1
 2  2  2  1  1  2
 2  2  1  1  1  1
 1  1  1  1  1  1
 2  2  2  2  2  2
 2  1  1  1  1  1

In [38]:
reset_messages!(bp)
reset_beliefs!(bp)
svd_trunc = TruncBond(10)

iters, = iterate!(bp, maxiter=5; svd_trunc, tol=1e-12, damp=0.5);

In [39]:
obs_node = 6
for t in eachindex(bp.w[obs_node])
    for j in eachindex(bp.w[obs_node][t].λ)
        bp.w[obs_node][t].λ[j] = λ[j,obs_node]
    end
end

der_mpbp = der_λ(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

2-element Vector{Float64}:
 -0.03738317757009529
  0.062305295950132615

In [40]:
ϵ = 1e-8

for i in vertices(g)
    for t in eachindex(bp.w[i])
        for j in eachindex(bp.w[i][t].λ)
            bp.w[i][t].λ[j] = λ[j,i]
        end
    end
end

logzᵢ0 = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

for t in eachindex(bp.w[obs_node])
    bp.w[obs_node][t].λ[1] = λ[1,obs_node] + ϵ
end

logzᵢϵ = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

der_num = logzᵢϵ/ϵ - logzᵢ0/ϵ 

-0.0373835563659668